In [1]:
import pandas as pd
import openpyxl
import re
import os
from pathlib import Path
import xlsxwriter
import requests
from requests_html import HTMLSession
s = HTMLSession()

## AccuWeather Hourly Weather Data - 1 Location

In [2]:
url = 'https://www.accuweather.com/en/kr/seoul/226081/hourly-weather-forecast/226081'   # Seoul Location
r = s.get(url, headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'})

# creating empty lists for storing hourly data
uv =[]; wind=[]; windgust=[]; humidity=[]; humidity_in=[]; airquality=[]; dewpoint=[]; cloudcover=[]; visibility=[]; cloudceiling=[]
time=[]; temp=[]; phrase=[] 
for i in range(0,5):                     # choose how many hours data you want to get...0 to 5 gives 5 hrs data 
    string = "div#hourlyCard"+str(i)
    print(string)

# while using html.find()  class is replaced with '.' and id is replaced with '#'
# If you want to find a class or id  which is inside a main div class, then it must be seperated by "space"

    Time = (r.html.find(f'{string} h2.date',first=True).text)

    time.append(Time) # time

    temp.append( (r.html.find(f'{string} div.temp.metric',first=True).text) )   #temp

    phrase.append( (r.html.find(f'{string} div.hourly-content-container.top-content-container div.phrase',first=True)).text ) #phrase

    panel1 = r.html.find(f'{string} div.hourly-content-container div.panel span.value') # gives a list of all the weather parameters
    

    if (( int(Time[:2])<8 and Time[-2:]=='AM' )  or ( int(Time[:2]) >5 and Time[-2:]=='PM' )):  # No UV index before 8am and after 5pm
         uv.append("None")
         wind.append(panel1[0].text)
         windgust.append(panel1[1].text)
         humidity.append(panel1[2].text)
         humidity_in.append(panel1[3].text)
         airquality.append(panel1[4].text)
         dewpoint.append(panel1[5].text)
         cloudcover.append(panel1[6].text)
         visibility.append(panel1[7].text)
        #cloudceiling.append(panel1[8].text)
        
        
    else:
        uv.append(panel1[0].text)
        wind.append(panel1[1].text)
        windgust.append(panel1[2].text)
        humidity.append(panel1[3].text)
        humidity_in.append(panel1[4].text)
        airquality.append(panel1[5].text)
        dewpoint.append(panel1[6].text)
        cloudcover.append(panel1[7].text)
        visibility.append(panel1[8].text)
        #cloudceiling.append(panel1[9].text)

div#hourlyCard0
div#hourlyCard1
div#hourlyCard2
div#hourlyCard3
div#hourlyCard4


In [3]:
dict = {'Time':time,'Temperature':temp, 'Phrase':phrase, 'UV Index':uv,  'Wind':wind, 'WindGust':windgust, 'humidity':humidity, "HumidityIndoor":humidity_in,
            'AirQuality':airquality, 'DewPoint':dewpoint,'CloudCover':cloudcover, 'Visibility':visibility }

In [4]:
df = pd.DataFrame(dict)
df

,Time,Temperature,Phrase,UV Index,Wind,WindGust,humidity,HumidityIndoor,AirQuality,DewPoint,CloudCover,Visibility
0,3 PM,13°,Showers,1 Low,NW 11 km/h,20 km/h,43%,41% (Slightly Dry),Fair,1° C,100%,16 km
1,4 PM,12°,Intermittent clouds,1 Low,NW 11 km/h,20 km/h,40%,39% (Slightly Dry),Fair,-1° C,59%,16 km
2,5 PM,12°,Partly sunny,0 Low,NW 11 km/h,20 km/h,39%,37% (Dry),Fair,-2° C,35%,16 km
3,6 PM,10°,Mostly clear,None,NW 9 km/h,20 km/h,40%,36% (Dry),Fair,-3° C,11%,16 km
4,7 PM,9°,Clear,None,NW 9 km/h,20 km/h,42%,36% (Dry),Fair,-3° C,9%,16 km


## AccuWeather Hourly Data - Multiple Locations

In [5]:
base = 'https://www.accuweather.com/en'   # base url 

city = ['Seoul','NewYork','Boston','Austin','Chicago','Canada']

cityid= {'Seoul': '/kr/seoul/', 'NewYork':'/us/new-york/','Boston':'/us/boston/','Austin':'/us/austin/','Chicago':'/us/chicago/','Canada':'/ca/regina/'}
id1 = {'Seoul': '226081', 'NewYork':'10021','Boston':'02108','Austin':'78701','Chicago':'60608','Canada':'s4p'} 
id2 = {'Seoul': '226081', 'NewYork':'349727','Boston':'348735','Austin':'351193','Chicago':'348308','Canada':'50335'}

# url = base + cityid + id1 + '/hourly-weather-forecast/' + id2
# if you want to get data of other locations, you need to get cityid, id1, id2 from "accuweather.com"
# Go to accuweather.com...enter desired city name in the search bar....copy the url and extract  id1 and id2 for desired location

In [6]:
for i in city:
    url = base + cityid[i] + id1[i] + '/hourly-weather-forecast/' + id2[i]
    r = s.get(url, headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'})
    print(url)

https://www.accuweather.com/en/kr/seoul/226081/hourly-weather-forecast/226081
https://www.accuweather.com/en/us/new-york/10021/hourly-weather-forecast/349727
https://www.accuweather.com/en/us/boston/02108/hourly-weather-forecast/348735
https://www.accuweather.com/en/us/austin/78701/hourly-weather-forecast/351193
https://www.accuweather.com/en/us/chicago/60608/hourly-weather-forecast/348308
https://www.accuweather.com/en/ca/regina/s4p/hourly-weather-forecast/50335


In [7]:
weather ={}
for i in city:
    url = base + cityid[i] + id1[i] + '/hourly-weather-forecast/' + id2[i]
    r = s.get(url, headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'})
    #print(url)
    #print(i)


    time=[]; location =[]; temp=[]; description=[]; uv=[]; wind=[]; windgust=[]; humidity=[]; humidity_in=[]; airquality=[]; dewpoint=[]; cloudcover=[]

    for j in range(0,15):             # number of hours data required
        try:
            string = "div#hourlyCard"+str(j)
            #print(string)

            Time = (r.html.find(f'{string} h2.date',first=True).text)

            location.append(i) 

            time.append(Time) # time

            temp.append( (r.html.find(f'{string} div.temp.metric',first=True).text) )   #temp

            description.append( (r.html.find(f'{string} div.hourly-content-container.top-content-container div.phrase',first=True)).text ) #phrase

            panel1 = r.html.find(f'{string} div.hourly-content-container div.panel span.value')  # gives list of all weather parameters
    

            if (( int(Time[:2])<8 and Time[-2:]=='AM' )  or ( int(Time[:2]) >5 and Time[-2:]=='PM' )):  # before 8am and after 5pm, no uv index parameter
         
                uv.append("None")
                wind.append(panel1[0].text)
                windgust.append(panel1[1].text)
                humidity.append(panel1[2].text)
                humidity_in.append(panel1[3].text)
                airquality.append(panel1[4].text)
                dewpoint.append(panel1[5].text)
                cloudcover.append(panel1[6].text)
        
            else:
                uv.append(panel1[0].text)
                wind.append(panel1[1].text)
                windgust.append(panel1[2].text)
                humidity.append(panel1[3].text)
                humidity_in.append(panel1[4].text)
                airquality.append(panel1[5].text)
                dewpoint.append(panel1[6].text)
                cloudcover.append(panel1[7].text)

        except AttributeError:    # if the index specified in range is not found, then finish the execution and go to next city in te list
            break
    
    dict = {'Time':time, 'Location':location, 'Temperature':temp, 'Weather':description, 'Wind':wind, 'WindGust':windgust, 'humidity':humidity, "HumidityIndoor":humidity_in,
            'AirQuality':airquality, 'DewPoint':dewpoint,'CloudCover':cloudcover }

    weather[i] = pd.DataFrame(dict)
    print(i)


Seoul
NewYork
Boston
Austin
Chicago
Canada


In [8]:
weather['Seoul']

,Time,Location,Temperature,Weather,Wind,WindGust,humidity,HumidityIndoor,AirQuality,DewPoint,CloudCover
0,3 PM,Seoul,13°,Showers,NW 11 km/h,20 km/h,43%,41% (Slightly Dry),Fair,1° C,100%
1,4 PM,Seoul,12°,Intermittent clouds,NW 11 km/h,20 km/h,40%,39% (Slightly Dry),Fair,-1° C,59%
2,5 PM,Seoul,12°,Partly sunny,NW 11 km/h,20 km/h,39%,37% (Dry),Fair,-2° C,35%
3,6 PM,Seoul,10°,Mostly clear,NW 9 km/h,20 km/h,40%,36% (Dry),Fair,-3° C,11%
4,7 PM,Seoul,9°,Clear,NW 9 km/h,20 km/h,42%,36% (Dry),Fair,-3° C,9%
5,8 PM,Seoul,8°,Clear,NW 9 km/h,20 km/h,45%,36% (Dry),Fair,-3° C,7%
6,9 PM,Seoul,8°,Clear,NW 7 km/h,20 km/h,47%,36% (Dry),Fair,-3° C,6%
7,10 PM,Seoul,7°,Clear,NW 7 km/h,20 km/h,50%,36% (Dry),Fair,-3° C,4%
8,11 PM,Seoul,6°,Clear,NW 7 km/h,20 km/h,52%,36% (Dry),Fair,-3° C,2%


In [9]:
weather['NewYork']

,Time,Location,Temperature,Weather,Wind,WindGust,humidity,HumidityIndoor,AirQuality,DewPoint,CloudCover
0,2 AM,NewYork,14°,Mostly clear,WNW 2 km/h,5 km/h,63%,42% (Ideal Humidity),Unhealthy,7° C,10%
1,3 AM,NewYork,13°,Mostly clear,WNW 4 km/h,5 km/h,64%,41% (Ideal Humidity),Unhealthy,7° C,10%
2,4 AM,NewYork,13°,Mostly clear,NW 4 km/h,5 km/h,66%,42% (Ideal Humidity),Unhealthy,7° C,10%
3,5 AM,NewYork,13°,Mostly clear,NNW 4 km/h,5 km/h,68%,42% (Ideal Humidity),Unhealthy,7° C,10%
4,6 AM,NewYork,12°,Mostly clear,NE 6 km/h,5 km/h,69%,41% (Ideal Humidity),Unhealthy,7° C,10%
5,7 AM,NewYork,12°,Mostly clear,NNE 6 km/h,5 km/h,70%,41% (Ideal Humidity),Unhealthy,6° C,10%
6,8 AM,NewYork,12°,Mostly sunny,NE 7 km/h,5 km/h,70%,42% (Ideal Humidity),Unhealthy,7° C,10%
7,9 AM,NewYork,13°,Sunny,NE 7 km/h,10 km/h,66%,41% (Ideal Humidity),Unhealthy,6° C,7%
8,10 AM,NewYork,14°,Sunny,ENE 7 km/h,10 km/h,58%,39% (Ideal Humidity),Unhealthy,6° C,3%
9,11 AM,NewYork,16°,Sunny,E 7 km/h,10 km/h,53%,40% (Ideal Humidity),Unhealthy,6° C,0%


In [10]:
print(weather)

{'Seoul':     Time Location Temperature              Weather        Wind WindGust  \
0   3 PM    Seoul         13°              Showers  NW 11 km/h  20 km/h   
1   4 PM    Seoul         12°  Intermittent clouds  NW 11 km/h  20 km/h   
2   5 PM    Seoul         12°         Partly sunny  NW 11 km/h  20 km/h   
3   6 PM    Seoul         10°         Mostly clear   NW 9 km/h  20 km/h   
4   7 PM    Seoul          9°                Clear   NW 9 km/h  20 km/h   
5   8 PM    Seoul          8°                Clear   NW 9 km/h  20 km/h   
6   9 PM    Seoul          8°                Clear   NW 7 km/h  20 km/h   
7  10 PM    Seoul          7°                Clear   NW 7 km/h  20 km/h   
8  11 PM    Seoul          6°                Clear   NW 7 km/h  20 km/h   

  humidity      HumidityIndoor AirQuality DewPoint CloudCover  
0      43%  41% (Slightly Dry)       Fair     1° C       100%  
1      40%  39% (Slightly Dry)       Fair    -1° C        59%  
2      39%           37% (Dry)       Fair    -2